# Convert and Process DTC data into netcdf files

Data loaded from science_teams DTC_CSV folder. 




## 1. Convert to xarray dataset 

1. load .csv file 
2. create standard dataset with same labels 
3. save as .nc file 

In [64]:
# load some standard packages 

import numpy as np
import xarray as xr
import math
import csv

import calendar

import pandas as pd

import gsw_xarray as gsw


In [21]:
#Input: 
# DTC_df: pandas DataFrame with columns 'Date' and 'Time' that need to be combined into a single datetime index
#output:
# DTC_df: pandas DataFrame with a single datetime index and no 'Date' or 'Time' columns
def index_pandas_with_datetime(DTC_df):

        # Convert the time column to string
        DTC_df['Time'] = DTC_df['Time'].astype(str)

        # Combine Date and Time into a single column
        DTC_df['Datetime'] = DTC_df['Date'] + ' ' + DTC_df['Time']

        # Function to parse datetime with optional milliseconds
        def parse_datetime(dt_str):
            for fmt in ('%d/%m/%y %H:%M:%S.%f', '%d/%m/%y %H:%M:%S'):
                try:
                    return pd.to_datetime(dt_str, format=fmt)
                except ValueError:
                    continue
            return pd.NaT  # Return Not-a-Time if parsing fails

        # Apply the function to the 'Datetime' column
        DTC_df['Datetime'] = DTC_df['Datetime'].apply(parse_datetime)

        # Drop rows where datetime parsing failed
        DTC_df = DTC_df.dropna(subset=['Datetime'])

        # Set the Datetime column as the index
        DTC_df = DTC_df.set_index('Datetime')

        # drop date and time columns
        DTC_df.drop(['Date', 'Time'], axis=1, inplace=True)

        
        return DTC_df

In [39]:
#Input:
# DTC_df: pandas DataFrame with columns 'Latitude' and 'Longitude' in the format 'ddmm.mmmmN' and 'dddmm.mmmmE'
#Output:
# DTC_df: pandas DataFrame with columns 'Latitude' and 'Longitude' converted to decimal degrees

def convert_to_decimal_degrees(value):
    # Check if the value is '<no response>'

    if value == '<no response>':
        return value
    
    # Convert the string to a float
    numeric_value = float(value)
    
    # Extract the degrees part (the first three digits)
    degrees = int(numeric_value // 100)
    
    # Extract the minutes part (the remaining part)
    minutes = numeric_value - (degrees * 100)
    
    # Convert minutes to decimal degrees
    decimal_degrees = degrees + (minutes / 60)

        
    return decimal_degrees


# Function to replace '<no response>' with NaN
def replace_no_response(value):
    if value == '<no response>':
        return pd.NA
    return value

In [59]:
# Input:    
# DTC_df: pandas DataFrame with columns 'Latitude' and 'Longitude' in the format 'ddmm.mmmmN' and 'dddmm.mmmmE'
# bottom_time: datetime object indicating the time when the bottom depth starts
#Output:
# DTC_ds: xarray Dataset with the following variables:
# lat: latitude in decimal degrees
# lon: longitude in decimal degrees
# depth: depth in meters
# temperature: CTD temperature
# conductivity: CTD conductivity
# pressure: CTD pressure
# oxygen: CTD oxygen
# salinity: CTD salinity
# coordinates: time - datetime index

def process_to_xr_dataset(DTC_df,bottom_time):


        # Apply the function to the entire DataFrame
    DTC_df = DTC_df.applymap(replace_no_response)

    # Drop rows with any NaN values
    DTC_df = DTC_df.dropna()

    # Reset index
    DTC_df = DTC_df.reset_index(drop=True)

    # convert to datetime
    DTC_df=index_pandas_with_datetime(DTC_df)

    # convert to decimal degrees

    lat_ms = DTC_df['Latitude'].values
    # Apply the conversion to each value in the list
    lat_degrees_list = [convert_to_decimal_degrees(value) for value in lat_ms]
    lat_degrees_list = [-x for x in lat_degrees_list] # in the southern hemisphere

    lon_ms = DTC_df['Longitude'].values
    # Apply the conversion to each value in the list
    lon_degrees_list = [convert_to_decimal_degrees(value) for value in lon_ms]

    # calculate depth
    depth_from_p=gsw.z_from_p(DTC_df['CTD Pressure'].values, lat_degrees_list, geo_strf_dyn_height=0, sea_surface_geopotential=0)

    # create xarray dataset
    ds = xr.Dataset(
    data_vars=dict(
        lat=(["time"], lat_degrees_list),
        lon=(["time"], lon_degrees_list),
        depth=(["time"], depth_from_p),
        temperature=(["time"], DTC_df['CTD Temperature']),
        conductivity=(["time"], DTC_df['CTD Conductivity']),
        pressure=(["time"], DTC_df['CTD Pressure']),
        oxygen=(["time"], DTC_df['CTD Oxygen']),
        salinity=(["time"], DTC_df['CTD Salinity']),
    ),
    coords=dict(
        time=("time", DTC_df.index)),
    attrs=dict(lat="Latitude in decimal degrees",
               lon="Longitude  in decimal degrees",
               depth="Calculated from pressure and latitude. Bottom depth starts at time "+str(bottom_time),
               conductivity="CTD condutivity S/m",
               pressure="CTD pressure dbar",
               oxygen="CTD oxygen ml/l",
               temperature="CTD temperature C",
               salinity="CTD salinity psu")

    )

    ds=ds.sortby('time')

    
    
    return ds

In [51]:
# Load the data

DTC_001 = pd.DataFrame(pd.read_csv("/Volumes/Science-Team/DTC_CSV/IN2024_V04_001/DTC_IN2024_V04_001_20240609T064927Z_ALL.csv"))

DTC_002 = pd.DataFrame(pd.read_csv("/Volumes/Science-Team/DTC_CSV/IN2024_V04_002/DTC_IN2024_V04_002_20240609T130743Z_ALL.csv"))

DTC_003 = pd.DataFrame(pd.read_csv("/Volumes/Science-Team/DTC_CSV/IN2024_V04_003/DTC_IN2024_V04_003_20240610T170229Z_ALL.csv"))

DTC_004 = pd.DataFrame(pd.read_csv("/Volumes/Science-Team/DTC_CSV/IN2024_V04_004/DTC_IN2024_V04_004_20240610T220841Z_ALL.csv"))

DTC_005 = pd.DataFrame(pd.read_csv("/Volumes/Science-Team/DTC_CSV/IN2024_V04_005/DTC_IN2024_V04_005_20240611T025208Z_ALL.csv"))



In [60]:
DTC_001_ds = process_to_xr_dataset(DTC_001, bottom_time = pd.to_datetime('2024-06-09 08:32:00'))
DTC_002_ds = process_to_xr_dataset(DTC_002, bottom_time = pd.to_datetime('2024-06-09 14:02:29'))
DTC_003_ds = process_to_xr_dataset(DTC_003, bottom_time = pd.to_datetime('2024-06-09 17:35:47'))
DTC_004_ds = process_to_xr_dataset(DTC_003, bottom_time = pd.to_datetime('2024-06-10 22:55:45'))
DTC_005_ds = process_to_xr_dataset(DTC_003, bottom_time = pd.to_datetime('2024-06-11 03:10:00'))



/var/folders/3m/hb6030nd3ml4fk_t3lq08f4w0000gn/T/ipykernel_47438/1571609333.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DTC_df = DTC_df.applymap(replace_no_response)
/var/folders/3m/hb6030nd3ml4fk_t3lq08f4w0000gn/T/ipykernel_47438/1571609333.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DTC_df = DTC_df.applymap(replace_no_response)
/var/folders/3m/hb6030nd3ml4fk_t3lq08f4w0000gn/T/ipykernel_47438/1571609333.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DTC_df = DTC_df.applymap(replace_no_response)
/var/folders/3m/hb6030nd3ml4fk_t3lq08f4w0000gn/T/ipykernel_47438/1571609333.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DTC_df = DTC_df.applymap(replace_no_response)
/var/folders/3m/hb6030nd3ml4fk_t3lq08f4w0000gn/T/ipykernel_47438/1571609333.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFr

In [61]:
DTC_005_ds

<xarray.Dataset>
Dimensions:       (time: 11690)
Coordinates:
  * time          (time) datetime64[ns] 2024-06-10T17:02:29.274000 ... 2024-0...
Data variables:
    lat           (time) float64 -30.09 -30.09 -30.09 ... -30.17 -30.17 -30.17
    lon           (time) float64 153.8 153.8 153.8 153.8 ... 153.7 153.7 153.7
    depth         (time) float64 -338.8 -339.3 -340.1 ... -95.75 -96.79 -96.72
    temperature   (time) float64 14.93 14.95 14.94 14.94 ... 24.29 24.3 24.3
    conductivity  (time) float64 4.325 4.325 4.324 4.323 ... 5.302 5.303 5.303
    pressure      (time) float64 341.4 342.0 342.7 343.6 ... 96.42 97.48 97.41
    oxygen        (time) float64 3.53 3.531 3.527 3.521 ... 4.178 4.185 4.19
    salinity      (time) float64 35.23 35.22 35.21 35.2 ... 35.49 35.49 35.49
Attributes:
    lat:           Latitude in decimal degrees
    lon:           Longitude  in decimal degrees
    depth:         Calculated from pressure and latitude. Bottom depth starts...
    conductivity:  CTD condutivity S/m
    pressure:      CTD pressure dbar
    oxygen:        CTD oxygen ml/l
    temperature:   CTD temperature C
    salinity:      CTD salinity psu

In [62]:
# save the data

DTC_001_ds.to_netcdf("/Volumes/Science-Team/DTC_CSV/IN2024_V04_001/DTC_IN2024_V04_001_20240609T064927Z_ALL.nc")
DTC_002_ds.to_netcdf("/Volumes/Science-Team/DTC_CSV/IN2024_V04_002/DTC_IN2024_V04_002_20240609T130743Z_ALL.nc")
DTC_003_ds.to_netcdf("/Volumes/Science-Team/DTC_CSV/IN2024_V04_003/DTC_IN2024_V04_003_20240610T170229Z_ALL.nc")
DTC_004_ds.to_netcdf("/Volumes/Science-Team/DTC_CSV/IN2024_V04_004/DTC_IN2024_V04_004_20240610T220841Z_ALL.nc")
DTC_005_ds.to_netcdf("/Volumes/Science-Team/DTC_CSV/IN2024_V04_005/DTC_IN2024_V04_005_20240611T025208Z_ALL.nc")
